In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from IPython.display import display

from math import sqrt
import csv



In [2]:
############# popularity for new users #############

In [3]:
def new_popularity(meta,ratings):
    avg = ratings.groupby('book_id')[['rating']].mean()
    count_rating = ratings.groupby('book_id')[['rating']].count()

    avg[avg['rating'].notnull()]['rating'] = avg[avg['rating'].notnull()]['rating'].astype('float')
    vote_averages= avg[avg['rating'].notnull()]['rating'] 
    C = vote_averages.mean()

    count_rating[count_rating['rating'].notnull()]['rating'] = count_rating[count_rating['rating'].notnull()]['rating'].astype('float')
    vote_counts = count_rating[count_rating['rating'].notnull()]['rating']
    m = vote_counts.quantile(0.9)

    meta['ratings_count'] = count_rating['rating']
    meta['average_rating'] = avg['rating']

    content = meta[(meta['ratings_count'].notnull())][['book_id','title','Genres', 'authors', 'ratings_count', 'average_rating']]

    content['ratings_count'] = content['ratings_count'].astype('float')

    content['average_rating'] = content['average_rating'].astype('float')

    #content.shape
    #print("v rating_count : " + str(ratings_count) + "R average_rating : " + str(average_rating)+"m" + str(m)+"C" + str(C))
    def weighted_rating(x):
        v = x['ratings_count']
        R = x['average_rating']
        return (v/(v+m) * R) + (m/(m+v) * C)

    content['popularity_rating'] = content.apply(weighted_rating, axis=1)
    p = content[['book_id','title','Genres']]
    pp = p[0:50]
    display(pp)


In [4]:
############ Popularity based #########################

In [5]:
def popularity(meta,ratings):
    avg = ratings.groupby('book_id')[['rating']].mean()
    count_rating = ratings.groupby('book_id')[['rating']].count()

    avg[avg['rating'].notnull()]['rating'] = avg[avg['rating'].notnull()]['rating'].astype('float')
    vote_averages= avg[avg['rating'].notnull()]['rating'] 
    C = vote_averages.mean()

    count_rating[count_rating['rating'].notnull()]['rating'] = count_rating[count_rating['rating'].notnull()]['rating'].astype('float')
    vote_counts = count_rating[count_rating['rating'].notnull()]['rating']
    m = vote_counts.quantile(0.9)

    meta['ratings_count'] = count_rating['rating']
    meta['average_rating'] = avg['rating']

    content = meta[(meta['ratings_count'].notnull())][['book_id','title', 'authors', 'ratings_count', 'average_rating']]

    content['ratings_count'] = content['ratings_count'].astype('float')

    content['average_rating'] = content['average_rating'].astype('float')

    content.shape
    #print("v rating_count : " + str(ratings_count) + "R average_rating : " + str(average_rating)+"m" + str(m)+"C" + str(C))
    def weighted_rating(x):
        v = x['ratings_count']
        R = x['average_rating']
        return (v/(v+m) * R) + (m/(m+v) * C)

    content['popularity_rating'] = content.apply(weighted_rating, axis=1)
    p = content[['book_id','popularity_rating']]
    #print(p)
    #print(pop.shape)
    print("popularity _done")
    return p

In [6]:
######################### COllaborative ############################## 

In [7]:
def collaborative(ratings,user_id):

    reader = Reader()
    #ratings.head()

    new_ratings = ratings
    data = Dataset.load_from_df(new_ratings[['user_id', 'book_id', 'rating']], reader)
    #data.split(n_folds=2)

    ## Training the data ##
    svd = SVD()
    cross_validate(svd, data, measures=['RMSE', 'MAE'],cv=2)

    trainset = data.build_full_trainset()

    algo = SVD()
    algo.fit(trainset)

    #svd.train(trainset)
    ## Testing the data ##
    testset = trainset.build_anti_testset()
    predictions = algo.test(testset)
    count = 0
     
    for uid, iid, true_r, est, _ in predictions:
        if uid == user_id:
            count = count+1
            new_ratings.loc[len(new_ratings)+1]= [uid,iid,est]
            #print("book_sim " + str(new_ratings))

    cb = new_ratings[(new_ratings['user_id'] == user_id)][['book_id', 'rating']]
    cb.columns=['book_id', 'collab_rating']
    #print(cb)
    print("Collab_done")
    return(cb)

In [8]:
################## content based #########################

In [9]:
def content_based(meta,ratings,user_id):       
    meta['book_id'] = meta['book_id'].astype('int')
    meta['authors'] = meta['authors'].str.replace(' ','')
    meta['authors'] = meta['authors'].str.lower()
    meta['authors'] = meta['authors'].str.replace(',',' ')
    ratings['book_id'] = ratings['book_id'].astype('int')
    ratings['user_id'] = ratings['user_id'].astype('int')
    ratings['rating'] = ratings['rating'].astype('int')


    #print(md.head())

    meta['authors'] = meta['authors'].apply(lambda x: [x,x])
    #print(md['authors'])
    meta['Genres']=meta['Genres'].str.split(';')
    #print(md['Genres'])

    meta['keywords'] = meta['authors'] + meta['Genres']
    #print(md['keywords'])
    meta['keywords'] = meta['keywords'].str.join(' ')

    tfidf = TfidfVectorizer(stop_words='english')

    tfidf_matrix = tfidf.fit_transform(meta['keywords'])
    tfidf_matrix.shape
    #print(tfidf_matrix)


    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    def build_user_profiles():
        
        user_profiles=np.zeros((60001,999))
        length = len(ratings)
        #for i in range(0,79999):
        for i in range(0,length):
            u=ratings.iloc[i]['user_id']
            u = round(u)
            b=ratings.iloc[i]['book_id']
            b = round(b)
            user_profiles[u][b-1]=ratings.iloc[i]['rating']
        #print(user_profiles)
        return user_profiles

    user_profiles=build_user_profiles()

    def get_similar_items_to_user_profile(person_id):
            

        user_ratings = np.empty((999,1))
        cnt=0
        
        for i in range(0,998):
            book_sim=cosine_sim[i]
            #print("book_sim " + str(book_sim))
            user_sim=user_profiles[person_id]
            #print("user_sim " + str(user_sim))
            user_ratings[i]=(book_sim.dot(user_sim))/sum(cosine_sim[i])
            #print("user_ratings " + str(user_ratings))
        maxval = max(user_ratings)
        #print(maxval)

        for i in range(0,998):
            user_ratings[i]=((user_ratings[i]*5.0)/(maxval))
            if(user_ratings[i]>3):
                cnt+=1
        #print(cnt)
        return user_ratings

    content_ratings = get_similar_items_to_user_profile(user_id)
    #print(content_ratings)
    #content_ratings = np.nan_to_num(content_ratings,copy = True,nan=0.0,posinf = None,neginf = None)

    book = meta[['book_id']]
    book1 = pd.DataFrame(data=content_ratings[0:,0:])
    frames = [book, book1]


    #content_rating = pd.concat(frames, axis =1,join_axes=[num.index])
    content_rating = pd.concat(frames, axis=1)
    content_rating = content_rating.reindex(book.index)
    content_rating.columns=['book_id', 'content_rating']
    
    
    #print(content_rating)
    print("content_done")
    return(content_rating)

In [10]:
################## Hybrid system #######################

In [11]:
def hybrid(user_id,ratings):
    meta = pd.read_csv('Dataset/metadata.csv')
    rat = ratings
    rat['rating'] = rat['rating'].astype('int')
    print(ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']])
    popularity_rating = popularity(meta,ratings)
    collaborative_rating = collaborative(ratings,user_id)
    #print(collaborative_rating)
    content_rating = content_based(meta,ratings,user_id)
            
    hyb = meta[['book_id']]
    title = meta[['book_id','title', 'Genres']]

    hyb = hyb.merge(title,on = 'book_id')
    hyb = hyb.merge(collaborative_rating,on = 'book_id')
    hyb = hyb.merge(popularity_rating, on='book_id')
    hyb = hyb.merge(content_rating, on='book_id')

    def weighted_rating(x):
        collab = x['collab_rating']
        pop = x['popularity_rating']
        content = x['content_rating']
        a = 0.6
        #res = (1-2*a)*content + a*collab + a*pop
        res = (2*a-1)*content + (2*a-1)*pop + a*collab
        #res = a*c + a*v + a*R
        res = "{:.4f}".format(res)
        return res

    hyb['hyb_rating'] = hyb.apply(weighted_rating, axis=1)
    hyb = hyb.sort_values('hyb_rating', ascending=False).head(999)
    hyb.columns = ['book_id' , 'title', 'genres', 'collaborative_rating', 'popularity_rating' , 'content_rating', 'hybrid_rating']
    
    #    y_true = rat[(rat['user_id'] == user_id)][['user_id','book_id', 'rating']]
    y_pred = hyb[['book_id','hybrid_rating']]
    hyb= hyb[0:100]

    rating = ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']]
    #rating = ratings[(ratings['user_id'] == user_id)][['user_id','book_id', 'rating']]
    y_pred = hyb[['book_id','hybrid_rating']]

    #y_pred = y_pred[0:100]
    hyb = hyb[0:100]
    #rint(y_pred)
    e = y_pred.merge(rating,on = 'book_id')
    e = e.iloc[: , 1:]
    #hyb.to_csv("prediction_list.csv")
    p = np.array(e[['hybrid_rating']])
    r = np.array(e[['rating']])
    err = sqrt(mean_squared_error(r, p))
    print("ERROR : " + str(err))
    print(hyb)    
    


In [16]:
def new_user():
    ratings = pd.read_csv('Dataset/ratings.csv')
    meta = pd.read_csv('Dataset/metadata.csv')
    user_id = 60000
    rating_count = len(ratings['user_id'])+1
    print('\n----------------Welcome User-------------------')
    print('\n-------------YOUR NEW USER ID '+str(user_id)+' -------------')
     
    print('\nPlease Rate Any 3(out of 5) books from the above list.')    

    ratings=ratings[1:100000]
    new_popularity(meta,ratings)
    

    

    
    for x in range(0,3):
        print("\n")
        bookId=input("BookId:")
       
        rating=input("Rating:")
        
        ratings.loc[rating_count]= [user_id,bookId,rating]
        rating_count =rating_count+1  
    h = hybrid(user_id,ratings)


In [17]:
print("--------------Welcome to the Book Recommendation Engine--------------\n")

user=input("1. New User. \n2. Existing User.\n")

if user=='1':
    new_user()
    
elif user=='2':
    ratings = pd.read_csv('Dataset/ratings.csv')
    ratings=ratings[1:100000]
    user_id=int(input("\nPlease Enter User Id: "))
    print('\n----------------Welcome User-------------------')
    h = hybrid(user_id,ratings)
    
    
    
else:
    print("Invalid option\n ")


--------------Welcome to the Book Recommendation Engine--------------

1. New User. 
2. Existing User.
1

----------------Welcome User-------------------

-------------YOUR NEW USER ID 60000 -------------

Please Rate Any 3(out of 5) books from the above list.


,book_id,title,Genres
2,3,"Twilight (Twilight, #1)",Fantasy
3,4,To Kill a Mockingbird,Self-Help;Drama
4,5,The Great Gatsby,Drama
5,6,The Fault in Our Stars,Romance;Drama
7,8,The Catcher in the Rye,Classic;Young-Age
8,9,"Angels & Demons (Robert Langdon, #1)",Thriller;Drama
9,10,Pride and Prejudice,Drama;Romance
10,11,The Kite Runner,Drama;History
11,12,"Divergent (Divergent, #1)",Fantasy;Drama
13,14,Animal Farm,Drama


KeyboardInterrupt: Interrupted by user

In [ ]:
# y_test = pd.read_csv('x_test.csv')
# cb = collaborative(y_test,3624)

In [ ]:
# new = y_test
# new['hybrid']='1'
# new.head(4)